In [14]:
import numpy as np
import itertools
import random

In [195]:
def heaviside(x):
    return np.array(1 * (x >= 0))
def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return list(itertools.chain.from_iterable(itertools.combinations(s, r) for r in range(len(s)+1)))

In [275]:
def extension_bool(p,V):
    """
    Compute a boolean vector that represents the extension of p in V
    
    Inputs:
        p - a vector of shape (M,) with elements from {-1,0,1}.  The partially specified feature vector
        V-  a matrix of shape (L,M) with elements from {-1,1}.  The feature vectors
    Outputs:
        extension - a vector of shape (L) with elements from {1,0}.  extension[l]=1 iff V[l,:] is in the extension of p
    """
    L = np.shape(V)[0]
    if np.sum(abs(p)) == 0:
        #the all zeros vector
        return np.ones(L)
    else:
        E = np.dot(V,p)/np.sum(abs(p))
        return heaviside(E-1)

def extension_multi_bool(p_mat,V):
    """
    Compute a boolean vector that represents the extension of p in V
    
    Inputs:
        p_mat - a matrix of shape (M,num_p) with elements from {-1,0,1}.  The matrix of partially specified
            feature vectors, containing num_p vectors
        V-  a matrix of shape (L,M) with elements from {-1,1}.  The feature vectors
    Outputs:
        extension - a matrix of shape (L,num_p) with elements from {1,0}.  extension[l,i]=1 iff V[l,:] is 
            in the extension of p_mat[:,i]
    """
    K_vec = np.sum(abs(p_mat),axis=0) #shape is (num_p,)
    E = np.dot(V,p_mat) #shape is (L,num_p)
    return heaviside(E-K_vec[np.newaxis,:])

def extension_mat(p,V):
    """
    Returns a matrix representing the extension of p in V
    
    Inputs:
        p - a vector of shape (M,) with elements from {-1,0,1}.  The partially specified feature vector
        V-  a matrix of shape (L,M) with elements from {-1,1}.  The feature vectors
    Outputs:
        A - A matrix of shape (N,M) with elements from {-1,1}, where N <=L, and the rows of A are the rows of V in the
            extension of p
    """
    
    bool_vec = extension_bool(p,V)
    return V[bool_vec>0,:]
def check_equivalent(p,q,V):
    """
    Checks if p and q have the same extension in V
    
    Inputs:
        p,q - vectors of shape (M,) with elements from {-1,0,1}.  The partially specified feature vectors
        V-  a matrix of shape (L,M) with elements from {-1,1}.  The feature vectors
    Outputs:
        match - bool, whether or not p and q have the same extension 
    """
    p_bool = extension_bool(p,V)
    q_bool = extension_bool(q,V)
    M = np.shape(q_bool)[0]
    return all([p_bool[i]==q_bool[i] for i in range(M)])

def mutate_p(p,abstraction_set,specification_set):
    """
    Returns q that is equal to p with mutations (specifications and generalizations applied)
    
    Inputs:
        p - the partially specified feature vector to be mutated
        abstraction_set - a set of indices to flip currently specified values of p to zero
        specification_set - a set of tuples (index,value) to change currently unspecified values of p to the given value
    Outputs:
        q - the mutated vector
    """
    
    q = np.copy(p)
    for abstraction in abstraction_set:
        q[abstraction]=0
    for specification_tuple in specification_set:
        q[specification_tuple[0]]=specification_tuple[1]
    return q

def get_allowed_specifications(p,V):
    """
    Given a set of feature vectors V and a query partially specified feature vector p, all locations where 
    p can be specified (flipped from a zero to +/- 1) without changing the extension in V.
    
    Inputs:
        p - a vector of shape (M,) with elements from {-1,0,1}.  The partially specified feature vector
        V -  a matrix of shape (L,M) with elements from {-1,1}.  The feature vectors
    Outputs:
        allowed_specifications - a list of tuples of form (index,value) where p[index] can be set to value
        without changing extension
    """
    
    M = np.shape(p)[0]
    
    bool_p = extension_bool(p,V)
    
    A = V[bool_p>0,:]#shape is (N,M)
    N = np.shape(A)[0]
    A_sums = np.sum(A,axis=0)
    
    allowed_specifications = []
    for i in range(M):
        if p[i]==0:
            #we can flip this position to a signed value if only if all A[:,i] or equivalently A_sums[i]== +/- N
            if N==0:
                #empty extension means we can replace all zeros with +1 or -1
                allowed_specifications.append((i,1))
                allowed_specifications.append((i,-1))              
            elif A_sums[i]==N:
                allowed_specifications.append((i,1)) #they're all 1 so we can mutate p[i] to 1
            elif A_sums[i]==-N:
                allowed_specifications.append((i,-1)) #they're all -1 so we can mutate p[i] to -1
    return allowed_specifications
    
def get_allowed_abstraction_sets(p,V):
    """
    Given a set of feature vectors V and a query partially specified feature vector p, return all partially
    specified vectors q that have the same extension in V as p.
    
    Inputs:
        p - a vector of shape (M,) with elements from {-1,0,1}.  The partially specified feature vector
        V -  a matrix of shape (L,M) with elements from {-1,1}.  The feature vectors
    Outputs:
        allowed_abstraction_sets - a list of sets of indices, where each set denotes a set of indices of p
            where a previously specified value can be flipped to zero without changing extension
    """
    bool_p = extension_bool(p,V)
    B = V[bool_p<=0,:]#shape is (T,M), p is not compatible with any vector in B
    (T,M) = np.shape(B)
    
    
    prespecified_indices = [i for i in range(M) if p[i] != 0]
    
    #now we find all subsets J of the specified indices such that if we zeroed out every index of p contained in J, the
    #new vector would be consistent with an element of B
    #this accounts to looking at each element b (row) of B, and finding all the indices where p disagrees 
    #with that element and adding that set of indices to an accumulating set
    forbidden_index_subsets_ = []
    
    for t_val in range(T):
        clash_points = []
        for m_val in prespecified_indices:
            if B[t_val,m_val]*p[m_val] == -1:
                clash_points.append(m_val)
        forbidden_index_subsets_.append(frozenset(clash_points))
    forbidden_index_subsets = set(forbidden_index_subsets_)
    
    #we can trim this down though by noting that any elment of forbidden_index_subsets that is a 
    #proper superset of another element of forbidden_index subsets is redundant
    """
    reduced_forbidden_index_subsets = set()
    for J in forbidden_index_subsets:
        retain = True
        for J_prime in forbidden_index_subsets:
            if J > J_prime: #J is a proper superset of J_prime
                retain=False
                break
        if retain:
            reduced_forbidden_index_subsets.add(J)
    """
    #now we try and build up all the legal subsets of the prespecified indices using a sort of breadth first
    #search.  We start with all the legal singletons
    
    allowed_abstraction_sets=set() #since not abstracting any indices is a legal mutation
    my_queue = []
    allowed_singletons = []
    for i in prespecified_indices:
        trial_singleton = frozenset([i])
        if not trial_singleton in forbidden_index_subsets:
            my_queue.append(trial_singleton)
            allowed_singletons.append(trial_singleton)
    
    #now we will build up allowed sets from elements of the queue
    
    while len(my_queue)>0:
        trial_set = my_queue.pop(0)
        if not (trial_set in forbidden_index_subsets):
            allowed_abstraction_sets.add(trial_set)
            #now we combine this allowed set with all the allowed singletons it doesn't contain and add to queue
            #for later processing
            for trial_singleton in allowed_singletons:
                if not (trial_set >= trial_singleton): #the trial set doesn't already contain this singleton
                    new_set = trial_set.union(trial_singleton)
                    my_queue.append(new_set)
            
    allowed_abstraction_sets.add(frozenset())
    return allowed_abstraction_sets#,forbidden_index_subsets
    
def equivalence_class(p,V):
    """
    Given a set of feature vectors V and a query partially specified feature vector p, return all partially
    specified vectors q that have the same extension in V as p.
    
    Inputs:
        p - a vector of shape (M,) with elements from {-1,0,1}.  The partially specified feature vector
        V-  a matrix of shape (L,M) with elements from {-1,1}.  The feature vectors
    Outputs:
        R - a list of vectors where R[i] has shape (M,) and all vectors in R have the same extension in V as p
    """
    
    M = np.shape(p)[0]
    L = np.shape(V)[0]
    V_sums = np.sum(V,axis=0)
    bool_p = extension_bool(p,V)
    
    allowed_specifications = get_allowed_specifications(p,V)
    allowed_specification_sets = powerset(allowed_specifications) #nope, can't have (i,1) (i,-1) in same
    allowed_abstraction_sets = get_allowed_abstraction_sets(p,V)
    
    equivalence_class = []
    
    for specification_set in allowed_specification_sets:
        for abstraction_set in allowed_abstraction_sets:
            #print abstraction_set
            q = mutate_p(p,abstraction_set,specification_set)
            equivalence_class.append(q)
    
    #return equivalence_class,allowed_abstraction_sets,allowed_specification_sets
    return equivalence_class
    #return allowed_specification_sets,legal_index_sets    

In [197]:
def test_eq(p,V,all_possible_p):
    eq_test = equivalence_class(p,V)
    bool_p = extension_bool(p,V)

    match_list = []
    for q in eq_test:
        bool_test = extension_bool(q,V)
        match_list.append(all(bool_p == bool_test))
    print all(match_list)
    eq_set = set([tuple(q) for q in eq_test])
    match_list = []
    for q in all_possible_p:
        if not q in eq_set:
            bool_test = extension_bool(np.array(q),V)
            match_list.append(all(bool_p==bool_test))
    print any(match_list)

In [305]:
def verbose_test(p,V,all_possible_p):
    bool_p = extension_bool(p,V)
    print('The boolean inclusion vector')
    print(bool_p)
    eq_test = equivalence_class(p,V)
    print('The proposed equivalence class')
    print(np.stack(eq_test,axis=0))
    eq_set = set([tuple(q) for q in eq_test])
    
    wrongly_included_q = [] #the elements of eq_test that shouldn't be there
    wrongly_excluded_q = [] #the psfvs not in eq_test that should be there
    
    for q in all_possible_p:
        bool_test = extension_bool(np.array(q),V)
        if q in eq_set:
            if not all(bool_p==bool_test):
                #shouldn't be there
                wrongly_included_q.append(q)
        else:
            if all(bool_p==bool_test):
                #it should be there
                wrongly_excluded_q.append(q)
    print('wrongly included psfvs')
    if len(wrongly_included_q)>0:
        print(np.stack(wrongly_included_q,axis=0))
    else:
        print('none')
    print('wrongly excluded psfvs')
    if len(wrongly_excluded_q)>0:
        print(np.stack(wrongly_excluded_q,axis=0))
    else:
        print('none')

In [105]:
M = 5
L = 10
all_feature_vecs = [comb for comb in itertools.product([-1,1],repeat=M)]
all_possible_p = [p for p in itertools.product([-1,0,1],repeat=M)]

In [150]:
v_list = random.sample(all_feature_vecs,L)
V = np.stack(v_list,axis=0)
V

array([[-1,  1, -1, -1, -1],
       [ 1, -1, -1,  1, -1],
       [-1, -1, -1, -1,  1],
       [ 1,  1, -1, -1, -1],
       [ 1,  1,  1,  1,  1],
       [-1,  1,  1, -1, -1],
       [-1,  1,  1, -1,  1],
       [ 1, -1,  1, -1, -1],
       [-1, -1,  1, -1,  1],
       [ 1,  1,  1, -1, -1]])

In [151]:
p_query = np.array(random.choice(all_possible_p))
p_query

array([ 1,  1,  0, -1, -1])

In [152]:
print(extension_bool(p_query,V))
ext_mat = extension_mat(p_query,V)
ext_mat

[0 0 0 1 0 0 0 0 0 1]


array([[ 1,  1, -1, -1, -1],
       [ 1,  1,  1, -1, -1]])

In [232]:
equivalence_class(p_query,V)

[array([0, 1, 0, 0, 1]),
 array([0, 1, 1, 0, 0]),
 array([-1,  1,  1,  0,  1]),
 array([ 0,  1,  1, -1,  1]),
 array([ 0,  1,  1, -1,  0]),
 array([-1,  1,  0,  0,  0]),
 array([ 0,  1,  0, -1,  0]),
 array([-1,  1,  1, -1,  0]),
 array([-1,  1,  0,  0,  1]),
 array([-1,  1,  0, -1,  1])]

In [318]:
p_query

array([-1,  1, -1,  0,  1])

In [319]:
V

array([[-1,  1,  1,  1, -1],
       [ 1, -1, -1, -1, -1],
       [ 1,  1,  1,  1,  1],
       [ 1, -1,  1,  1, -1],
       [ 1,  1, -1,  1, -1],
       [-1,  1, -1,  1, -1],
       [-1,  1, -1, -1,  1],
       [-1, -1,  1,  1, -1],
       [-1, -1,  1, -1, -1],
       [-1,  1, -1, -1, -1]])

In [317]:
verbose_test(p_query,V,all_possible_p)

The boolean inclusion vector
[0 0 0 0 0 0 1 0 0 0]
The proposed equivalence class
[[ 0  1 -1  0  1]
 [-1  0  0  0  1]
 [-1  1 -1  0  1]
 [-1  0 -1  0  1]
 [ 0  0  0  0  1]
 [-1  1  0  0  1]
 [ 0  0 -1  0  1]
 [ 0  1 -1 -1  1]
 [-1  0  0 -1  1]
 [-1  1 -1 -1  1]
 [-1  0 -1 -1  1]
 [ 0  0  0 -1  1]
 [-1  1  0 -1  1]
 [ 0  0 -1 -1  1]]
wrongly included psfvs
[[0 0 0 0 1]]
wrongly excluded psfvs
[[ 0  1  0 -1  1]]


In [316]:
v_list = random.sample(all_feature_vecs,L)
V = np.stack(v_list,axis=0)
p_query = np.array(random.choice(all_possible_p))
test_eq(p_query,V,all_possible_p)

False
True


In [309]:
V

array([[-1,  1, -1, -1, -1],
       [-1,  1,  1, -1,  1],
       [ 1,  1, -1, -1,  1],
       [-1, -1,  1,  1,  1],
       [ 1,  1, -1,  1,  1],
       [ 1,  1,  1, -1,  1],
       [ 1,  1, -1, -1, -1],
       [ 1,  1,  1,  1,  1],
       [-1, -1, -1, -1,  1],
       [-1,  1, -1, -1,  1]])

In [292]:
extension_mat(p_query,V)

array([], shape=(0, 5), dtype=int64)

In [293]:
p_query

array([ 1, -1,  1, -1,  0])

In [294]:
get_allowed_abstraction_sets(p_query,V)

{frozenset(), frozenset({0})}

In [299]:
get_allowed_specifications(p_query,V)

[(4, 1), (4, -1)]

In [298]:
equivalence_class(p_query,V)

[array([ 1, -1,  1, -1,  0]),
 array([ 0, -1,  1, -1,  0]),
 array([ 1, -1,  1, -1,  1]),
 array([ 0, -1,  1, -1,  1]),
 array([ 1, -1,  1, -1, -1]),
 array([ 0, -1,  1, -1, -1]),
 array([ 1, -1,  1, -1, -1]),
 array([ 0, -1,  1, -1, -1])]

In [274]:
a = set()
a.add(frozenset())
len(a)

1